Create environment as below:
From anaconda prompt: `create -n tensorflow_env python=3.5 anaconda`

`activate tensorflow_env`

`conda install theano`

`conda install mingw libpython`

`pip install tensorflow`

`pip install keras` 

Then run `spyder` or `jupyter notebook`

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Artificial Neural Networks 

### <font color='red'> Data preprocessing </font>

#### <font color='blue'> Data input </font>

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#dataset.shape

(10000, 14)

In [3]:
X = dataset.iloc[:, 3: 13]
y = dataset.iloc[:, 13]

In [5]:
#print(X.shape, y.shape)

(10000, 10) (10000,)


In [6]:
#X[0:3]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57


In [7]:
#X.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

#### <font color='blue'> Categorical variables </font>

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder_gender = LabelEncoder()
X.iloc[:, 2] = label_encoder_gender.fit_transform(X.iloc[:, 2])
X= pd.DataFrame(X)
X[0:3]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57


In [5]:
X.rename(columns={'Gender':'Male'}, inplace=True)

In [10]:
# X[0:3]

,CreditScore,Geography,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57


In [6]:
X= pd.get_dummies(X, columns=['Geography'])

X[0:3]

,CreditScore,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0


In [7]:
# Drop the dummy column for France
X = X.drop(['Geography_France'], axis=1)

In [9]:
#X[0:3]

,CreditScore,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0


####  <font color='blue'> Train test split </font>

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
#type(X_train)

pandas.core.frame.DataFrame

#### <font color='blue'> Feature scaling </font>

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
#X_train[0:3]

array([[ 0.16958176, -1.09168714, -0.46460796,  0.00666099, -1.21571749,
         0.8095029 ,  0.64259497, -1.03227043,  1.10643166, -0.5698444 ,
         1.74309049],
       [-2.30455945,  0.91601335,  0.30102557, -1.37744033, -0.00631193,
        -0.92159124,  0.64259497,  0.9687384 , -0.74866447,  1.75486502,
        -0.57369368],
       [-1.19119591, -1.09168714, -0.94312892, -1.031415  ,  0.57993469,
        -0.92159124,  0.64259497, -1.03227043,  1.48533467, -0.5698444 ,
        -0.57369368]])

### <font color='red'> Building the ANN </font>

In [20]:
#import keras
#from keras.models import Sequential
#from keras.layers import Dense 

In [21]:
# Initializing the ANN 
#classifier = Sequential()
#classifier.add(Dense(6, input_dim = 11, kernel_initializer = 'uniform', activation='relu')) # Choosing six nodes in hidden layer, as avg of 11 (input) and 1 (output)
#classifier.add(Dense(6, kernel_initializer = 'uniform', activation='relu'))
#classifier.add(Dense(1, kernel_initializer = 'uniform', activation='sigmoid'))

In [22]:
#classifier.compile(loss= 'binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [23]:
#classifier.summary()

In [24]:
#classifier.fit(X_train, y_train, batch_size=10, epochs=100)

In [25]:
#epochs = classifier.history.epoch
#acc = classifier.history.history['acc']
#loss = classifier.history.history['loss']

In [26]:
# Making predictions and evaluating the model #
#y_pred = classifier.predict_classes(X_test)

In [27]:
from sklearn.metrics import confusion_matrix

#cm = confusion_matrix(y_test, y_pred)
#print(cm, cm[0,0])

In [28]:
#y_test.value_counts()

In [29]:
#cm_df = pd.DataFrame({'Predicted 0': [cm[0,0], cm[1,0]],
            'Predicted 1':[cm[0,1], cm[1,1]]},
                    index = ['Actual 0','Actual 1'])
#print(cm_df)

IndentationError: unexpected indent (<ipython-input-29-8b3c77215ff8>, line 2)

In [ ]:
#pd.Series(acc).plot(label='Accuracy', legend=True)
#pd.Series(loss).plot(secondary_y=True, label='Loss', legend=True)
#plt.show()

In [ ]:
# eg_customer = X[X.index==0] # Extracting the first row as a dataframe
# eg_customer.CreditScore = 600
# X.Male = 1
# X.Age = 40
# X.Tenure = 3
# X.Balance = 60000
# X.NumOfProducts = 2
# X.HasCrCard = 1
# X.IsActiveMember = 1 
# X.EstimatedSalary = 50000
# X.Geography_Germany=0
# X.Geography_Spain=0
# eg_customer

In [ ]:
# customer_pred = classifier.predict_classes(eg_customer)
# customer_pred

__Conclusion__ : We don't give this customer a loan


### <font color='red'>  Model evaluation </font>

#### <font color='green'> k-fold cross validation </font>

In [30]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier # Wrapper integrates the sklearn kfold CV with the Keras classifier

C:\Users\sridevi.tolety\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
def build_classifier():
    import keras
    from keras.models import Sequential
    from keras.layers import Dense
    classifier = Sequential()
    classifier.add(Dense(6, input_dim=11, activation='relu', kernel_initializer='uniform'))
    classifier.add(Dense(6, activation='relu', kernel_initializer='uniform'))
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier #Return the local classifier

In [32]:
# Global classifier 
classifier = KerasClassifier(build_fn = build_classifier, epochs=100, batch_size=10)

# Cross val score 
# if __name__ == "__main__":
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, verbose=1, n_jobs=1)

Epoch 1/100
7200/7200 [==============================] - 2s 266us/step - loss: 0.4857 - acc: 0.7961
Epoch 2/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.4284 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 2s 216us/step - loss: 0.4232 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 2s 231us/step - loss: 0.4190 - acc: 0.8164
Epoch 5/100
7200/7200 [==============================] - 1s 208us/step - loss: 0.4165 - acc: 0.8257
Epoch 6/100
7200/7200 [==============================] - 1s 204us/step - loss: 0.4147 - acc: 0.8297
Epoch 7/100
7200/7200 [==============================] - 1s 178us/step - loss: 0.4130 - acc: 0.8315
Epoch 8/100
7200/7200 [==============================] - 1s 165us/step - loss: 0.4122 - acc: 0.8315
Epoch 9/100
7200/7200 [==============================] - 1s 165us/step - loss: 0.4109 - acc: 0.8326
Epoch 10/100
7200/7200 [==============================] - 1s 170us/step - loss: 0.4093 - acc: 0.8339

7200/7200 [==============================] - 2s 256us/step - loss: 0.3568 - acc: 0.8533
Epoch 63/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.3556 - acc: 0.8556
Epoch 64/100
7200/7200 [==============================] - 2s 232us/step - loss: 0.3545 - acc: 0.8558
Epoch 65/100
7200/7200 [==============================] - 1s 195us/step - loss: 0.3542 - acc: 0.8574
Epoch 66/100
7200/7200 [==============================] - 2s 225us/step - loss: 0.3516 - acc: 0.8562 1s
Epoch 67/100
7200/7200 [==============================] - 2s 211us/step - loss: 0.3517 - acc: 0.8562
Epoch 68/100
7200/7200 [==============================] - 2s 251us/step - loss: 0.3509 - acc: 0.8578
Epoch 69/100
7200/7200 [==============================] - 2s 212us/step - loss: 0.3506 - acc: 0.8575
Epoch 70/100
7200/7200 [==============================] - 2s 218us/step - loss: 0.3488 - acc: 0.8569
Epoch 71/100
7200/7200 [==============================] - 2s 221us/step - loss: 0.3490 - acc: 0.8581


7200/7200 [==============================] - 2s 245us/step - loss: 0.3486 - acc: 0.8567
Epoch 22/100
7200/7200 [==============================] - 2s 234us/step - loss: 0.3485 - acc: 0.8587
Epoch 23/100
7200/7200 [==============================] - 2s 227us/step - loss: 0.3474 - acc: 0.8562
Epoch 24/100
7200/7200 [==============================] - 2s 211us/step - loss: 0.3464 - acc: 0.8590
Epoch 25/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3456 - acc: 0.8603
Epoch 26/100
7200/7200 [==============================] - 1s 198us/step - loss: 0.3460 - acc: 0.8593
Epoch 27/100
7200/7200 [==============================] - 1s 172us/step - loss: 0.3457 - acc: 0.8592
Epoch 28/100
7200/7200 [==============================] - 1s 165us/step - loss: 0.3458 - acc: 0.8569
Epoch 29/100
7200/7200 [==============================] - 1s 170us/step - loss: 0.3450 - acc: 0.8614
Epoch 30/100
7200/7200 [==============================] - 1s 185us/step - loss: 0.3447 - acc: 0.8587
Epo

7200/7200 [==============================] - 1s 155us/step - loss: 0.3957 - acc: 0.8353
Epoch 83/100
7200/7200 [==============================] - 1s 140us/step - loss: 0.3959 - acc: 0.8343
Epoch 84/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.3960 - acc: 0.8347
Epoch 85/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.3960 - acc: 0.8356
Epoch 86/100
7200/7200 [==============================] - 1s 148us/step - loss: 0.3955 - acc: 0.8346
Epoch 87/100
7200/7200 [==============================] - 1s 143us/step - loss: 0.3959 - acc: 0.8347 0s - loss: 0.4017 - ac
Epoch 88/100
7200/7200 [==============================] - 1s 154us/step - loss: 0.3958 - acc: 0.8324
Epoch 89/100
7200/7200 [==============================] - 1s 136us/step - loss: 0.3951 - acc: 0.8353
Epoch 90/100
7200/7200 [==============================] - 1s 158us/step - loss: 0.3958 - acc: 0.8343
Epoch 91/100
7200/7200 [==============================] - 1s 136us/step - loss: 0

7200/7200 [==============================] - 1s 139us/step - loss: 0.3840 - acc: 0.8321
Epoch 5/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.3777 - acc: 0.8325
Epoch 6/100
7200/7200 [==============================] - 1s 137us/step - loss: 0.3718 - acc: 0.8443
Epoch 7/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.3686 - acc: 0.8483
Epoch 8/100
7200/7200 [==============================] - 1s 137us/step - loss: 0.3663 - acc: 0.8497
Epoch 9/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.3638 - acc: 0.8521
Epoch 10/100
7200/7200 [==============================] - 1s 139us/step - loss: 0.3616 - acc: 0.8519
Epoch 11/100
7200/7200 [==============================] - 1s 143us/step - loss: 0.3600 - acc: 0.8519
Epoch 12/100
7200/7200 [==============================] - 1s 136us/step - loss: 0.3579 - acc: 0.8529
Epoch 13/100
7200/7200 [==============================] - 1s 137us/step - loss: 0.3566 - acc: 0.8560
Epoch 14

7200/7200 [==============================] - 1s 141us/step - loss: 0.3350 - acc: 0.8615
Epoch 86/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3355 - acc: 0.8635
Epoch 87/100
7200/7200 [==============================] - 1s 137us/step - loss: 0.3350 - acc: 0.8607
Epoch 88/100
7200/7200 [==============================] - 1s 141us/step - loss: 0.3345 - acc: 0.8617
Epoch 89/100
7200/7200 [==============================] - 1s 136us/step - loss: 0.3348 - acc: 0.8601
Epoch 90/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3351 - acc: 0.8607
Epoch 91/100
7200/7200 [==============================] - 1s 140us/step - loss: 0.3347 - acc: 0.8604
Epoch 92/100
7200/7200 [==============================] - 1s 136us/step - loss: 0.3356 - acc: 0.8628
Epoch 93/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3353 - acc: 0.8631
Epoch 94/100
7200/7200 [==============================] - 1s 143us/step - loss: 0.3347 - acc: 0.8611
Epo

7200/7200 [==============================] - 1s 150us/step - loss: 0.4039 - acc: 0.8342
Epoch 45/100
7200/7200 [==============================] - 1s 149us/step - loss: 0.4037 - acc: 0.8337
Epoch 46/100
7200/7200 [==============================] - 1s 146us/step - loss: 0.4047 - acc: 0.8336
Epoch 47/100
7200/7200 [==============================] - 1s 148us/step - loss: 0.4048 - acc: 0.8343
Epoch 48/100
7200/7200 [==============================] - 1s 149us/step - loss: 0.4043 - acc: 0.8315
Epoch 49/100
7200/7200 [==============================] - 1s 147us/step - loss: 0.4042 - acc: 0.8329
Epoch 50/100
7200/7200 [==============================] - 1s 148us/step - loss: 0.4044 - acc: 0.8347
Epoch 51/100
7200/7200 [==============================] - 1s 150us/step - loss: 0.4046 - acc: 0.8337
Epoch 52/100
7200/7200 [==============================] - 1s 151us/step - loss: 0.4043 - acc: 0.8353
Epoch 53/100
7200/7200 [==============================] - 1s 149us/step - loss: 0.4042 - acc: 0.8335
Epo

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 20.5min finished


In [43]:
mean = accuracies.mean()
variance = accuracies.std()
print("Mean is {:.3%} and stdev is {:.3%}".format(mean, variance))

Mean is 84.612% and stdev is 1.411%


### <font color='red'> Model improvements </font>


#### <font color='green'> Dropout regularization </font>

In [46]:
from keras.models import Sequential
from keras.layers import Dropout, Dense

In [48]:
classifier = Sequential()
classifier.add(Dense(6, input_dim=11, kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(p=0.1)) #Fraction of inputs to dropout. Keep increasing till overfitting improves
classifier.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

C:\Users\sridevi.tolety\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(X_train, y_train, epochs=100, batch_size=10)

#### <font color='green'> Hyperparameter tuning </font>

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
# Grid of parameters to tune 
parameters = {'batch_size':[10, 25, 32],
              'epochs':[50, 100],
             'optimizer':['adam', 'rmsprop']}

In [ ]:
# The funciton to feed into the sklearn keras classifier wrapper
def build_classifier(optimizer):
    print("Testing batch size={}, epochs={} and optimizer={}".format(batch_size, epochs, optimizer))
    classifier = Sequential()
    classifier.add(Dense(output_dim=6, input_dim=11, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(output_dim=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(output_dim=1, activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier)

In [45]:
grid_search = GridSearchCV(estimator=classifier, scoring='accuracy', param_grid=parameters, cv=10,verbose=1)

In [47]:
grid_search = grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


C:\Users\sridevi.tolety\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=6, input_dim=11, activation="relu")`
  after removing the cwd from sys.path.
C:\Users\sridevi.tolety\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=6, activation="relu")`
  """
C:\Users\sridevi.tolety\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  


Epoch 1/50
7200/7200 [==============================] - 4s 618us/step - loss: 0.4679 - acc: 0.7969
Epoch 2/50
7200/7200 [==============================] - 2s 276us/step - loss: 0.4274 - acc: 0.7971
Epoch 3/50
7200/7200 [==============================] - 2s 245us/step - loss: 0.4206 - acc: 0.8108
Epoch 4/50
7200/7200 [==============================] - 2s 289us/step - loss: 0.4170 - acc: 0.8210
Epoch 5/50
7200/7200 [==============================] - 2s 291us/step - loss: 0.4154 - acc: 0.8289
Epoch 6/50
7200/7200 [==============================] - 2s 260us/step - loss: 0.4140 - acc: 0.8287
Epoch 7/50
7200/7200 [==============================] - 2s 262us/step - loss: 0.4122 - acc: 0.8308
Epoch 8/50
7200/7200 [==============================] - 2s 260us/step - loss: 0.4111 - acc: 0.8310
Epoch 9/50
7200/7200 [==============================] - 2s 280us/step - loss: 0.4103 - acc: 0.8317
Epoch 10/50
7200/7200 [==============================] - 2s 303us/step - loss: 0.4093 - acc: 0.8322
Epoch 11/

7200/7200 [==============================] - 2s 242us/step - loss: 0.3498 - acc: 0.8567
Epoch 14/50
7200/7200 [==============================] - 2s 237us/step - loss: 0.3496 - acc: 0.8553
Epoch 15/50
7200/7200 [==============================] - 2s 251us/step - loss: 0.3486 - acc: 0.8568
Epoch 16/50
7200/7200 [==============================] - 2s 241us/step - loss: 0.3471 - acc: 0.8582
Epoch 17/50
7200/7200 [==============================] - 2s 250us/step - loss: 0.3456 - acc: 0.8586
Epoch 18/50
7200/7200 [==============================] - 2s 249us/step - loss: 0.3448 - acc: 0.8568
Epoch 19/50
7200/7200 [==============================] - 2s 250us/step - loss: 0.3446 - acc: 0.8594
Epoch 20/50
7200/7200 [==============================] - 2s 320us/step - loss: 0.3427 - acc: 0.8583
Epoch 21/50
7200/7200 [==============================] - 2s 250us/step - loss: 0.3431 - acc: 0.8587
Epoch 22/50
7200/7200 [==============================] - 2s 241us/step - loss: 0.3421 - acc: 0.8578
Epoch 23/50


7200/7200 [==============================] - 2s 246us/step - loss: 0.3963 - acc: 0.8382
Epoch 46/50
7200/7200 [==============================] - 2s 243us/step - loss: 0.3960 - acc: 0.8379
Epoch 47/50
7200/7200 [==============================] - 2s 246us/step - loss: 0.3957 - acc: 0.8368
Epoch 48/50
7200/7200 [==============================] - 2s 245us/step - loss: 0.3956 - acc: 0.8367
Epoch 49/50
7200/7200 [==============================] - 2s 248us/step - loss: 0.3954 - acc: 0.8371
Epoch 50/50
7200/7200 [==============================] - 2s 242us/step - loss: 0.3956 - acc: 0.8374
Epoch 1/50
7200/7200 [==============================] - 4s 555us/step - loss: 0.4866 - acc: 0.7942
Epoch 2/50
7200/7200 [==============================] - 2s 248us/step - loss: 0.4273 - acc: 0.7944
Epoch 3/50
7200/7200 [==============================] - 2s 246us/step - loss: 0.4192 - acc: 0.8185
Epoch 4/50
7200/7200 [==============================] - 2s 243us/step - loss: 0.4131 - acc: 0.8304
Epoch 5/50
7200/

...

7200/7200 [==============================] - 1s 188us/step - loss: 0.4026 - acc: 0.8349
Epoch 62/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4027 - acc: 0.8340
Epoch 63/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.4024 - acc: 0.8347
Epoch 64/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.4026 - acc: 0.8364
Epoch 65/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.4025 - acc: 0.8346
Epoch 66/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4025 - acc: 0.8344
Epoch 67/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4025 - acc: 0.8358
Epoch 68/100
7200/7200 [==============================] - 1s 187us/step - loss: 0.4022 - acc: 0.8342
Epoch 69/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.4023 - acc: 0.8347
Epoch 70/100
7200/7200 [==============================] - 1s 180us/step - loss: 0.4023 - acc: 0.8350
Epo

7200/7200 [==============================] - 1s 192us/step - loss: 0.3537 - acc: 0.8532
Epoch 23/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3523 - acc: 0.8535
Epoch 24/100
7200/7200 [==============================] - 1s 185us/step - loss: 0.3514 - acc: 0.8558
Epoch 25/100
7200/7200 [==============================] - 1s 207us/step - loss: 0.3499 - acc: 0.8556
Epoch 26/100
7200/7200 [==============================] - 2s 238us/step - loss: 0.3487 - acc: 0.8528
Epoch 27/100
7200/7200 [==============================] - 1s 185us/step - loss: 0.3482 - acc: 0.8550
Epoch 28/100
7200/7200 [==============================] - 1s 200us/step - loss: 0.3471 - acc: 0.8585
Epoch 29/100
7200/7200 [==============================] - 1s 191us/step - loss: 0.3463 - acc: 0.8571
Epoch 30/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.3449 - acc: 0.8553
Epoch 31/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.3441 - acc: 0.8599
Epo

7200/7200 [==============================] - 1s 190us/step - loss: 0.4048 - acc: 0.8326
Epoch 83/100
7200/7200 [==============================] - 1s 187us/step - loss: 0.4047 - acc: 0.8339
Epoch 84/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4046 - acc: 0.8326
Epoch 85/100
7200/7200 [==============================] - 1s 185us/step - loss: 0.4047 - acc: 0.8344
Epoch 86/100
7200/7200 [==============================] - 1s 178us/step - loss: 0.4045 - acc: 0.8335
Epoch 87/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4046 - acc: 0.8343
Epoch 88/100
7200/7200 [==============================] - 1s 191us/step - loss: 0.4044 - acc: 0.8324
Epoch 89/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4044 - acc: 0.8339
Epoch 90/100
7200/7200 [==============================] - 1s 190us/step - loss: 0.4044 - acc: 0.8332
Epoch 91/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4043 - acc: 0.8339
Epo

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 259.3min finished


Epoch 1/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5225 - acc: 0.7958
Epoch 2/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.4297 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 239us/step - loss: 0.4235 - acc: 0.7986
Epoch 4/100
8000/8000 [==============================] - 2s 241us/step - loss: 0.4183 - acc: 0.8231
Epoch 5/100
8000/8000 [==============================] - 2s 238us/step - loss: 0.4146 - acc: 0.8290
Epoch 6/100
8000/8000 [==============================] - 2s 238us/step - loss: 0.4114 - acc: 0.8309
Epoch 7/100
8000/8000 [==============================] - 2s 241us/step - loss: 0.4091 - acc: 0.8327
Epoch 8/100
8000/8000 [==============================] - 2s 243us/step - loss: 0.4068 - acc: 0.8344
Epoch 9/100
8000/8000 [==============================] - 2s 241us/step - loss: 0.4053 - acc: 0.8346
Epoch 10/100
8000/8000 [==============================] - 2s 240us/step - loss: 0.4036 - acc: 0.8340
E

In [54]:
best_params = grid_search.best_params_
best_acc = grid_search.best_score_

In [52]:
best_params

{'batch_size': 25, 'epochs': 100, 'optimizer': 'adam'}

In [55]:
best_acc

0.845625

## Convolutional Neural Networks

Input image -> Convolve over many feature maps -> Convolutional Layer -> Rectifier function 

-> Pooling

-> Flattenning

-> Fully connected

ReLU - Adds non-linearity

### <font color='red'> Data preprocessing </font>

Folder structure:
1. Train data: 
    * Cats: 4000
    * Dogs: 4000
2. Test data:
    * Cats: 1000
    * Dogs: 1000

Go through this and implement: 

https://github.com/tflearn/tflearn/blob/master/examples/basics/use_dask.py

In [1]:
# import findspark
# findspark.init()

In [2]:
# import pyspark
# from pyspark import SparkContext, SparkConf
# conf = SparkConf().setAppName('Elephas_App').setMaster('local[8]')
# sc =  SparkContext(conf=conf)

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten 

C:\Users\sridevi.tolety\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
classifier = Sequential()

#### Step 1: Convolution 

In [3]:
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), input_shape=(64, 64, 3), activation='relu')) 
# input_shape: (3, 256, 256) means 3 channels (for RGB) with 256*256 pixels each 
# But this is the order of Theano backend, whereas we are using Tensorflow backend 
# So make is (256, 256, 3) 

#### Step 2: Pooling 

In [4]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

#### Step 3: Flattening 

In [5]:
classifier.add(Flatten())

#### Step 4: Full connection

In [6]:
classifier.add(Dense(units=128, activation='relu')) # units is output_dim
classifier.add(Dense(units=1, activation='sigmoid'))

In [7]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,937,409
Trainable params: 3,937,409
Non-trainable params: 0
_________________________________________________________________


In [8]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Fitting the CNN to the images 

https://keras.io/preprocessing/image/


Enrich and augment our dataset to reduce overfitting by adding more images 

In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(
        rescale=1./255, #to make all pixel values between 0 and 1 
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
training_set = train_datagen.flow_from_directory('CNN_dataset/training_set',
                                                    target_size=(64, 64),#dimensions expected by 64 
                                                    batch_size=32,
                                                    class_mode='binary')

Found 8000 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory('CNN_dataset/test_set',
                                                        target_size=(64, 64),
                                                        batch_size=32,
                                                        class_mode='binary')

Found 2000 images belonging to 2 classes.


** create an RDD from numpy arrays**

In [ ]:
# from elephas.utils.rdd_utils import to_simple_rdd
# rdd = to_simple_rdd(sc, training_set, test_set)

In [ ]:
# from elephas.spark_model import SparkModel

# spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
# # spark_model.fit(rdd, epochs=1, batch_size=32, verbose=0, validation_split=0.1)

In [15]:
classifier.fit_generator(training_set,
                    steps_per_epoch=8000,
                    epochs=1,
                    validation_data=test_set,
                    validation_steps=800)

Epoch 1/1
8000/8000 [==============================] - 37350s 5s/step - loss: 0.3847 - acc: 0.8199 - val_loss: 0.8025 - val_acc: 0.7320


In [ ]:
classifier.fit_generator(training_set,
                    steps_per_epoch=8000,
                    epochs=1,
                    validation_data=test_set,
                    validation_steps=800)

Epoch 1/1
7488/8000 [===========================>..] - ETA: 42:11 - loss: 0.3951 - acc: 0.8141

### Making new predictions 

In [16]:
import numpy as np 
from keras.preprocessing import image

In [18]:
test_image = image.load_img('CNN_dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))
#target size should be same dim as training data

test_image = image.img_to_array(test_image) # create a 3-dim array from original 2-dim image 
test_image = np.expand_dims(test_image, axis=0) # add one more dimension corresponding to the batch (single input not accepted)
# Without this, will get error saying 4 dims expected 
result = classifier.predict(test_image)

In [19]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [20]:
if result[0][0]==1:
    prediction="dog"
else:
    prediction="cat"

In [21]:
prediction

'dog'

In [22]:
test_image = image.load_img('CNN_dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)

if result[0][0]==1:
    prediction="dog"
else:
    prediction="cat"
    
prediction

'dog'

high accuracy solution, see below:

https://www.udemy.com/deeplearning/learn/v4/questions/2276518

### Evaluating, improving and tuning the CNN 